In [1]:
import pandas as pd
import numpy as np
import math

In [118]:
athletes = pd.read_csv("data/olympic_athletes.csv", sep = ';')
hosts = pd.read_csv("data/olympic_hosts.csv", sep = ',')
medals = pd.read_csv("data/olympic_medals.csv", sep = ';')
results = pd.read_csv("data/olympic_results.csv", sep = ';')

In [119]:
def country_unifier(row):
    if row == "People's Republic of China":
        return 'China'
    if row == "Hong Kong, China":
        return 'Hong Kong'
    if row == 'United States':
        return 'United States of America'
    if row == 'Australia, Sweden':
        return 'Australia'
    if row == 'USSR':
        return 'Soviet Union'
    else:
        return row

## Nodes

### 1) Hosts

In [120]:
# Unify country names.
hosts.game_location = hosts.game_location.apply(lambda x: country_unifier(x))

# Save data in a csv file.
hosts.to_csv('csv/Hosts.csv', index = False, header=['slug', 'end_date', 'start_date', 'location', 'name', 'season', 'year'])

### 2) Countries

In [78]:
# Get only list of names without url from results.athletes.
def find_tuple_athl_url(row):
    if row is not np.nan:
        split_row = row.split("'")[1::2]
        tuple_list = []
        for name in split_row[::2]:
            tuple_list.append(name)
    else:
        return np.nan
    
    return tuple_list

# Remove urls and get list of names.
results.athletes = results.athletes.apply(lambda x: find_tuple_athl_url(x))

# Unify country names.
results.country_name = results.country_name.apply(lambda x: country_unifier(x))

In [79]:
# Create country instances.
countries = results[['country_name', 'country_code']].dropna().drop_duplicates()

# Save data in a csv file.
countries.to_csv('csv/Countries.csv', index = False, header=['name', 'code'])

### 3) Athletes

In [125]:
athletes.head(2)

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,G,S,B
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000,0,0,0
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995,0,0,0


In [124]:
def nan_to_0(row):
    if math.isnan(row):
        return 0
    else:
        return int(row)
    
def nan_to_NA(row):
    if type(row) == str:
        return row
    elif math.isnan(row):
        return 'NA'

def nan_to_NA_or_int(row):
    if math.isnan(row):
        return 'NA'
    elif isinstance(row, float):
        return int(row)

# Set nan of number of medals as 0.
athletes.G = athletes.G.apply(lambda x: nan_to_0(x))
athletes.S = athletes.S.apply(lambda x: nan_to_0(x))
athletes.B = athletes.B.apply(lambda x: nan_to_0(x))

# Replace nan with 'NA' and code as int athlete_year_birth.
athletes.first_game = athletes.first_game.apply(lambda x: nan_to_NA(x)) 
athletes.athlete_year_birth = athletes.athlete_year_birth.apply(lambda x: nan_to_NA_or_int(x)) 

In [128]:
# Save data in a csv file.
athletes.to_csv('csv/Athletes.csv', index = False, header=['url', 'name', 'partecipations', 'first_game', 'birth', 'G', 'S', 'B'])

## Edges

### 4) TAKE_PLACE_IN

In [117]:
# Take column to create edges.
TAKE_PLACE_IN = hosts[['game_slug', 'game_location']]

# Save in a csv file.
TAKE_PLACE_IN.to_csv('csv/TAKE_PLACE_IN.csv', index = False, header=['slug', 'location'])

### 5) NATIONALITY

## Athletes dataframe

In [87]:
athletes.head(2)

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,G,S,B
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000,0.0,0.0,0.0
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995,0.0,0.0,0.0


In [98]:
# Insert attributes only if are not null.
with open('insert_athletes.txt', 'w', encoding="utf-8") as f:
    for row in athletes.iterrows():
        url, name, partecipations, first_game, birth, G, S, B = row[1]
        attrs = f'''name:"{name}"'''

        new_row = f'''CREATE (:Athlete \u007B{attrs}\u007D)\n'''
    
        f.write(new_row)

## Medals dataframe

In [88]:
# Unify country names.
medals.country_name = medals.country_name.apply(lambda x: country_unifier(x))

# Remove duplicate column (check country_name).
medals = medals.drop('participant_title', axis=1)

In [89]:
medals.head(2)

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,athlete,country_name,country_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Stefania CONSTANTINI,Italy,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Amos MOSANER,Italy,ITA


## Results dataframe

In [72]:
results.head(2)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_position,country_name,country_code
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",1,Italy,ITA
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",2,Norway,NOR


In [85]:
# TAKE_PLACE_IN.
with open('TAKE_PLACE_IN.txt', 'w', encoding="utf-8") as f:
    for row in hosts.iterrows():
        slug, end_date, start_date, location, name, season, year = row[1]
        new_edge = f'''MATCH (h:Host), (c:Country) WITH h, c WHERE h.game_slug = "{slug}" AND c.name ="{location}" CREATE (h)-[:TAKE_PLACE_IN]->(c);\n'''
        f.write(new_edge)